In [16]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.spatial import KDTree

class GeometricPhaseCalibrator:
    def __init__(self, calibration_table_path='calibration_table_geometric.csv'):
        """
        基于几何夹角的相位差校准器
        校准表格式: [Raw_Ch0-3, Calibrated_Ch0-3, 理论夹角0-3]
        """
        self.calibration_table = np.loadtxt(calibration_table_path, delimiter=',', skiprows=1)
        
        # 提取数据
        self.raw_phases = self.calibration_table[:, 0:4]      # 原始相位差
        self.calibrated_phases = self.calibration_table[:, 4:8] # 校准后相位差
        self.theoretical_angles = self.calibration_table[:, 8:12] # 各通道理论夹角
        
        # 构建KDTree（基于理论夹角）
        self.kdtree = KDTree(self.theoretical_angles)
        
        print("几何校准器初始化完成")
        print(f"校准表包含 {len(self.calibration_table)} 个数据点")
        print(f"理论夹角维度: {self.theoretical_angles.shape[1]}")
    
    def calculate_theoretical_angles(self, azimuth, elevation, nab_vectors):
        """
        计算理论夹角（基于几何关系）
        
        Parameters:
        azimuth: 方位角（度）
        elevation: 俯仰角（度）
        nab_vectors: 天线对向量数组 [N01, N12, N23, N30] 的形状 (4, 3)
        
        Returns: 各天线对的理论夹角数组 [θ01, θ12, θ23, θ30]
        """
        # 将角度转换为弧度
        az_rad = np.deg2rad(azimuth)
        el_rad = np.deg2rad(elevation)
        
        # 计算目标方向向量（从基站指向目标）
        target_direction = np.array([
            np.cos(el_rad) * np.cos(az_rad),
            np.cos(el_rad) * np.sin(az_rad),
            np.sin(el_rad)
        ])
        
        theoretical_angles = []
        
        # 计算每对天线的理论夹角
        for i in range(len(nab_vectors)):
            nab_vector = nab_vectors[i]
            
            # 计算夹角（使用点积公式）
            cos_theta = np.dot(nab_vector, target_direction) / (
                np.linalg.norm(nab_vector) * np.linalg.norm(target_direction)
            )
            theta = np.arccos(np.clip(cos_theta, -1, 1))
            theoretical_angles.append(np.rad2deg(theta))
        
        return np.array(theoretical_angles)
    
    def calibrate_with_geometry(self, raw_phases, theoretical_angles):
        """
        使用理论夹角进行校准
        
        Parameters:
        raw_phases: 原始相位差测量值
        theoretical_angles: 计算得到的理论夹角
        
        Returns: 校准后的相位差
        """
        # 找到最接近的理论夹角点
        distance, idx = self.kdtree.query(theoretical_angles, k=1)
        return self.calibrated_phases[idx]
    
    def find_nearest_calibration_point(self, theoretical_angles):
        """
        查找最接近的校准点信息（用于调试）
        """
        distance, idx = self.kdtree.query(theoretical_angles, k=1)
        
        return {
            'index': idx,
            'distance': distance,
            'theoretical_angles_match': self.theoretical_angles[idx],
            'raw_phases_match': self.raw_phases[idx],
            'calibrated_phases_match': self.calibrated_phases[idx]
        }

def generate_geometric_calibration_table(original_calibration_table, nab_vectors):
    """
    将原有的方位角校准表转换为几何夹角校准表
    
    Parameters:
    original_calibration_table: 原有的校准表 [Raw_Ch0-3, Calibrated_Ch0-3, Azimuth]
    nab_vectors: 天线对向量数组 [N01, N12, N23, N30] 的形状 (4, 3)
    
    Returns: 几何夹角校准表 [Raw_Ch0-3, Calibrated_Ch0-3, Theoretical_Angles0-3]
    """
    geometric_table = []
    
    for i in range(len(original_calibration_table)):
        azimuth = original_calibration_table[i, 8]  # 方位角
        elevation = 0  # 假设圆周运动时俯仰角为0
        
        # 计算理论夹角
        theoretical_angles = calculate_theoretical_angles_simple(azimuth, elevation, nab_vectors)
        
        # 创建新行：原始值 + 校准值 + 理论夹角
        new_row = np.concatenate([
            original_calibration_table[i, 0:4],   # Raw_Ch0-3
            original_calibration_table[i, 4:8],   # Calibrated_Ch0-3
            theoretical_angles                    # Theoretical_Angles0-3
        ])
        
        geometric_table.append(new_row)
    
    return np.array(geometric_table)

def calculate_theoretical_angles_simple(azimuth, elevation, nab_vectors):
    """
    简化的理论夹角计算（独立函数）
    """
    # 将角度转换为弧度
    az_rad = np.deg2rad(azimuth)
    el_rad = np.deg2rad(elevation)
    
    # 计算目标方向向量
    target_direction = np.array([
        np.cos(el_rad) * np.cos(az_rad),
        np.cos(el_rad) * np.sin(az_rad),
        np.sin(el_rad)
    ])
    
    theoretical_angles = []
    
    # 计算每对天线的理论夹角
    for nab_vector in nab_vectors:
        cos_theta = np.dot(nab_vector, target_direction) / (
            np.linalg.norm(nab_vector) * np.linalg.norm(target_direction)
        )
        theta = np.arccos(np.clip(cos_theta, -1, 1))
        theoretical_angles.append(np.rad2deg(theta))
    
    return np.array(theoretical_angles)

# 批量校准函数
def batch_calibrate_with_geometry(z_data, azimuth_data, elevation_data, nab_vectors):
    """
    批量校准数据
    
    Parameters:
    z_data: 二维数组，每行 [相位差0, 相位差1, 相位差2, 相位差3, 距离]
    azimuth_data: 方位角数组
    elevation_data: 俯仰角数组
    nab_vectors: 天线对向量数组
    
    Returns: 校准后的数据数组
    """
    calibrator = GeometricPhaseCalibrator()
    calibrated_data = []
    
    for i in range(len(z_data)):
        # 计算理论夹角
        theoretical_angles = calibrator.calculate_theoretical_angles(
            azimuth_data[i], elevation_data[i], nab_vectors
        )
        
        # 进行校准
        calibrated = calibrator.calibrate_with_geometry(z_data[i, :4], theoretical_angles)
        calibrated_data.append(np.concatenate([calibrated, [z_data[i, 4]]]))
    
    return np.array(calibrated_data)

In [17]:

# # 使用示例
# if __name__ == "__main__":

DeltaThetaDeg = [0, 90, 180,270]
dAntenna = 0.0204  # 天线间距
kAntennaPairCount = len(DeltaThetaDeg)
# 天线基线向量（根据实际硬件修改）
Nab = np.zeros((2, kAntennaPairCount))  # 天线向量
Nab[0, 0] = 0
Nab[1, 0] = dAntenna

kAntennaCount = Nab.shape[1]

for col in range(1, kAntennaPairCount):
    theta = DeltaThetaDeg[col] / 180 * np.pi
    Nab[:, col] = np.dot(
        [[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]], Nab[:, 0]
    )
Nab = Nab.T
print(Nab)

nab_vectors  = np.c_[Nab, np.zeros(Nab.shape[0])]
print(nab_vectors)



[[ 0.00000000e+00  2.04000000e-02]
 [-2.04000000e-02  1.24913974e-18]
 [-2.49827947e-18 -2.04000000e-02]
 [ 2.04000000e-02 -3.74741921e-18]]
[[ 0.00000000e+00  2.04000000e-02  0.00000000e+00]
 [-2.04000000e-02  1.24913974e-18  0.00000000e+00]
 [-2.49827947e-18 -2.04000000e-02  0.00000000e+00]
 [ 2.04000000e-02 -3.74741921e-18  0.00000000e+00]]


In [18]:
# 1. 生成几何校准表
print("生成几何校准表...")
original_table = np.loadtxt('calibration_table_4channels_with_two.csv', delimiter=',', skiprows=1)
geometric_table = generate_geometric_calibration_table(original_table, nab_vectors)

# 保存新的几何校准表
header = 'Raw_Ch0,Raw_Ch1,Raw_Ch2,Raw_Ch3,Calibrated_Ch0,Calibrated_Ch1,Calibrated_Ch2,Calibrated_Ch3,Theta0,Theta1,Theta2,Theta3'
np.savetxt('calibration_table_geometric.csv', geometric_table, 
        delimiter=',', header=header, comments='', fmt='%.6f')

print("几何校准表生成完成！")

# 2. 使用几何校准器
calibrator = GeometricPhaseCalibrator()

# 测试数据（不同俯仰角）
azimuth_test = 45.0    # 方位角
elevation_test = 30.0  # 俯仰角（与校准时的0度不同！）
raw_phases_test = np.array([25.3, -18.7, 42.1, -12.5])

# 计算理论夹角
theoretical_angles = calibrator.calculate_theoretical_angles(
    azimuth_test, elevation_test, nab_vectors
)

# 进行校准
calibrated_phases = calibrator.calibrate_with_geometry(raw_phases_test, theoretical_angles)

# 查看匹配信息
match_info = calibrator.find_nearest_calibration_point(theoretical_angles)

print(f"\n测试点信息:")
print(f"方位角: {azimuth_test}°, 俯仰角: {elevation_test}°")
print(f"理论夹角: {theoretical_angles}")
print(f"原始相位: {raw_phases_test}")
print(f"校准相位: {calibrated_phases}")

print(f"\n匹配的校准点:")
print(f"索引: {match_info['index']}")
print(f"距离: {match_info['distance']:.4f}")
print(f"匹配理论夹角: {match_info['theoretical_angles_match']}")
print(f"匹配原始相位: {match_info['raw_phases_match']}")
print(f"匹配校准相位: {match_info['calibrated_phases_match']}")



生成几何校准表...
几何校准表生成完成！
几何校准器初始化完成
校准表包含 834 个数据点
理论夹角维度: 4

测试点信息:
方位角: 45.0°, 俯仰角: 30.0°
理论夹角: [ 52.23875609 127.76124391 127.76124391  52.23875609]
原始相位: [ 25.3 -18.7  42.1 -12.5]
校准相位: [ 114.907193  123.295016 -131.423344 -106.491518]

匹配的校准点:
索引: 529
距离: 14.4775
匹配理论夹角: [ 45. 135. 135.  45.]
匹配原始相位: [ 114.116278  143.746893 -137.109294 -120.753877]
匹配校准相位: [ 114.907193  123.295016 -131.423344 -106.491518]
